In [1]:
import os
import tensorflow as tf
import deepxde as dde
import numpy as np

# Set TensorFlow to use CPU only


os.environ['DDE_BACKEND'] = 'tensorflow'
# Define the spatial and time domains
geom = dde.geometry.Interval(0, 1)
time_domain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, time_domain)

# Define the PDE function
def pde(x, y):
    u = y
    u_t = dde.grad.jacobian(y, x, i=0, j=1)
    u_xx = dde.grad.hessian(y, x, i=0, j=0)
    return u_t - u_xx

# Define the boundary condition function
def boundary_func(x):
    return 0

# Define the initial condition function
def initial_func(x):
    return np.sin(np.pi * x[:, 0:1])

# Set up the boundary condition
bc = dde.icbc.DirichletBC(geomtime, boundary_func, lambda _, on_boundary: on_boundary)

# Set up the initial condition
ic = dde.icbc.IC(geomtime, initial_func, lambda _, on_initial: on_initial)

# Define observation data (for supervised learning)
observe_x = np.vstack((np.linspace(0, 0.05, num=10), np.full((10), 1))).T
observe_y = dde.icbc.PointSetBC(observe_x, np.sin(np.pi * observe_x[:, 0:1]))

# Set up the data object
data = dde.data.TimePDE(geomtime, pde, [ic, bc, observe_y], num_domain=40, num_test=10000)

# Multi-CPU configuration
strategy = tf.distribute.MirroredStrategy(devices=["/cpu:0", "/cpu:1"])  # Specify CPU devices

with strategy.scope():
    # Set up the model and compile
    net = dde.maps.FNN([2] + [32] * 3 + [1], "tanh", "Glorot uniform")
    model = dde.Model(data, net)
    
    # Compile the model with specific optimizer settings
    model.compile("adam", lr=0.001)
    
    # Train the model
    losshistory, train_state = model.train(epochs=10000)

    # Save the results
    dde.saveplot(losshistory, train_state, issave=True, isplot=True)


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.



Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Using MirroredStrategy with devices ('/replica:0/task:0/device:CPU:0', '/replica:0/task:0/device:CPU:1')
Compiling model...
Building feed-forward neural network...



c:\Users\yesda\anaconda3\Lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'build' took 0.242058 s




ValueError: DistributedVariable.handle is not available outside the replica context or a `tf.distribute.Strategy.update()` call.